In [1]:
import pandas as pd
import json
import os

from dotenv import load_dotenv
load_dotenv()

from openai import AzureOpenAI
from pydantic import BaseModel, HttpUrl

In [2]:
df = pd.read_csv("raw_data/Business_Licenses_20250621.csv") 
df.head()

,APN,RecordID,BusDesc,B1_PER_SUB_TYPE,DBA,NAICS,Tax_Code,Employee_Num,Bus_Own_Type,B1_BUSINESS_NAME,...,B1_ADDRESS2,B1_CITY,B1_STATE,B1_ZIP,B1_CONTACT_TYPE,B1_FULL_ADDRESS,B1_SITUS_CITY,B1_SITUS_STATE,B1_SITUS_ZIP,Business_Location
0,055 187800400,BL-032324,RESTAURANT,Retail Trade,KIPS RESTAURANT,722110,R,14,Corporation,KIPS RESTAURANT,...,NaN,BERKELEY,CA,947041610,Business Owner,2439 DURANT AVE,BERKELEY,CA,94704.0,NaN
1,057 205900600,BL-018844,JAPANESE FOOD & DRINKS,Retail Trade,KINDA SAKE BAR,722513 - Limited-Service Restaurants,R,16,Corporation,KINDA SAKE BAR,...,NaN,EL CERRITO,CA,94530-2811,Business Owner,1941 UNIVERSITY Ave,BERKELEY,CA,94704.0,NaN
2,057 205300302,BL-021115,RESTAURANT,Retail Trade,VIETNAM HOUSE,722513 - Limited-Service Restaurants,R,2,LLC,VIETNAM HOUSE,...,NaN,ALAMEDA,CA,94501-4410,Business Owner,1986 SHATTUCK Ave,BERKELEY,CA,94708.0,NaN
3,056 194600604,BL-048399,RESTAURANT,Retail Trade,"VIK'S CHAAT & MARKET, LLC",722211,R,26,LLC,"VIK'S CHAAT & MARKET, LLC",...,NaN,BERKELEY,CA,94710-2402,Business Owner,2390 FOURTH ST,BERKELEY,CA,94710.0,NaN
4,052 153100900,BL-054096,ARCHITECT,Professional SemiProfessional,DONAHUE RICHARD S,541310 - Architectural Services,P,1,Sole Ownership,DONAHUE RICHARD S,...,NaN,BERKELEY,CA,94703,Business Owner,1743 ALCATRAZ AVE,BERKELEY,CA,94703.0,NaN


In [3]:
# build the schema for businesses
from pydantic import BaseModel, HttpUrl, Field
from typing import List, Dict, Optional


class Hours(BaseModel):
    mon_sun: str = Field(..., min_length=1)


class Vibe(BaseModel):
    crowd: str
    atmosphere: str
    events: List[str]


class Reviews(BaseModel):
    yelp_rating: float
    restaurantguru_rating: float
    birdeye_rating: float
    common_feedback: List[str]
    sample_quotes: List[str]


class SocialMedia(BaseModel):
    instagram: str
    latest_event_post: str


class Business(BaseModel):
    name: str
    address: str
    phone: str
    website: HttpUrl
    hours: Hours
    established: Optional[str]
    type: str
    menu_highlights: List[str]
    vibe: Vibe
    reviews: Reviews
    parking: str
    payment: List[str]
    wifi: str
    delivery: str
    social_media: SocialMedia


business_info_schema = {
    "type": "function",
    "function": {
        "name": "get_business_info",
        "description": "Structured business metadata for businesses in Berkeley, CA.",
        "parameters": Business.model_json_schema()
    }
}


In [5]:
shortened_df = df[["BusDesc", "B1_BUSINESS_NAME", "B1_FULL_ADDRESS"]]

endpoint = os.getenv("ENDPOINT_URL", "https://2025-ai-hackberkeley.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "o4-mini")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")

client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2024-12-01-preview",  # Use a valid API version
)

def get_business_info(business_name):
    response = client.chat.completions.create(
                model=deployment,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a search agent, and your task is to collect more information based on a given information of a business. use web search. access reviews about the business if you can. access big chunk of texts regarding what they serve or who they serve for. include things such as vibes about the place that cannot be easily found through traditional searh engines. The user will provide the business information, and you will limit your answer to a json data format with clear data output regarding any information you can find.",
                    },
                    {
                        "role": "user",
                        "content": f"{business_name}",
                    },
                ],
                tools=[business_info_schema],
            )
    
    raw_output = response.choices[0].message.content
    cleaned = raw_output.strip().removeprefix("```json").removesuffix("```").strip()

    return cleaned

In [ ]:
from tqdm import tqdm
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# Wrap your function to safely handle errors and return index
def safe_call(index, row):
    query = row['B1_BUSINESS_NAME'] + " " + row['B1_FULL_ADDRESS']
    print(f"Processing {query}...")
    try:
        response = get_business_info(query)
        print(f"Response for {query}: {response}")
        return index, json.dumps(response)  # serialize to store safely in CSV
    except Exception as e:
        print(f"Error processing {query}: {e}")
        return index, None

# Run concurrently
with ThreadPoolExecutor(max_workers=32) as executor:  # tune max_workers as needed
    futures = [
        executor.submit(safe_call, idx, row)
        for idx, row in shortened_df.iterrows()
    ]

    for future in tqdm(as_completed(futures), total=len(futures)):
        idx, result = future.result()
        shortened_df.at[idx, 'Business Detailed Data'] = result

shortened_df.to_csv("augmented_data/business_info.csv", index=False)

Processing KIPS RESTAURANT 2439 DURANT AVE...Processing KINDA SAKE BAR 1941 UNIVERSITY Ave...

Processing VIETNAM HOUSE 1986 SHATTUCK Ave...
Processing VIK'S CHAAT & MARKET, LLC 2390 FOURTH ST...
Processing DONAHUE RICHARD S 1743 ALCATRAZ AVE...
Processing GLAM COUTURE 3205 SACRAMENTO ST...
Processing CHIPOTLE MEXICAN GRILL #697 2311 TELEGRAPH AVE...
Processing SAHIN EMIRHAN 0 VARIOUS...
Processing HOFFMAN CAROL L 1510 WALNUT ST...
Processing BEN HARDY & COMPANY 0 VARIOUS...
Processing ASHA TEA HOUSE 2086 UNIVERSITY AVE...
Processing RASPUTIN RECORDS 2401 TELEGRAPH AVE...
Processing TANE VEGAN 1956 SHATTUCK Ave...
Processing HAPPY GRILL LLC 0 VARIOUS...
Processing PARKER STREET FOUNDATION 2330 PARKER ST...
Processing XULO 2326 FIFTH St...
Processing MARREE 1314 ALCATRAZ Ave...
Processing KAIZEN CHIROPRACTIC + WELLNESS 2459 ASHBY Ave...
Processing PING KUEN CONST CO 0 VARIOUS...
Processing SHIBUMI GALLERY LLC 1816 FOURTH St...
Processing ASHKAR OMAR 1776 UNIVERSITY Ave...
Processing HES

  0%|          | 0/12335 [00:00<?, ?it/s]/var/folders/kc/nx_bxmjn3q130mt2rjp_5vsh0000gn/T/ipykernel_94459/398640200.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shortened_df.at[idx, 'Business Detailed Data'] = result
  0%|          | 1/12335 [00:06<23:40:56,  6.91s/it]

Error processing HOFFMAN CAROL L 1510 WALNUT ST: 'NoneType' object has no attribute 'strip'Error processing ASHKAR OMAR 1776 UNIVERSITY Ave: 'NoneType' object has no attribute 'strip'
Processing F D THOMAS INC 0 VARIOUS...

Processing YEGANEH REZA 1504 BONITA AVE...


  0%|          | 3/12335 [00:07<6:36:12,  1.93s/it] 

Response for PING KUEN CONST CO 0 VARIOUS: {
  "name": "PING KUEN CONST CO",
  "address": null,
  "phone": null,
  "website": null,
  "hours": {
    "mon_sun": ""
  },
  "established": null,
  "type": null,
  "menu_highlights": [],
  "vibe": {
    "crowd": "",
    "atmosphere": "",
    "events": []
  },
  "reviews": {
    "yelp_rating": 0,
    "restaurantguru_rating": 0,
    "birdeye_rating": 0,
    "common_feedback": [],
    "sample_quotes": []
  },
  "parking": "",
  "payment": [],
  "wifi": "",
  "delivery": "",
  "social_media": {
    "instagram": "",
    "latest_event_post": ""
  }
}
Processing OXTAIL STUDIO LLC 2976 ADELINE ST...
Error processing SHIBUMI GALLERY LLC 1816 FOURTH St: 'NoneType' object has no attribute 'strip'
Processing TRAX 1812 FIFTH ST...


...

Error processing SANCHEZ ZARRIN 1808 M L KING JR Way: 'NoneType' object has no attribute 'strip'
Processing HELFAND GARY 2729 GRANT St...
Error processing ACME SCENERY COMPANY INC 2451 FIFTH St: 'NoneType' object has no attribute 'strip'
Processing AGHAMIR S & RAZAZAN T 3145 COLLEGE AVE...


 47%|████▋     | 5823/12335 [31:48<20:25,  5.31it/s]

Error processing KAUFMAN CONSTRUCTION INC 0 VARIOUS: 'NoneType' object has no attribute 'strip'
Processing CHEW ANDREW & LILLIE 2229 MCGEE AVE...
Error processing CASTLE SINICROPE 1425 MILVIA St: 'NoneType' object has no attribute 'strip'
Processing GREEN TILE GROUP LLC 2913 SHATTUCK Ave...


 47%|████▋     | 5824/12335 [31:49<33:28,  3.24it/s]

Error processing SIMPLY SALMA'S TREASURES 0 VARIOUS: 'NoneType' object has no attribute 'strip'
Processing MATOSSIAN JOSEPH 1887 SOLANO AVE...
